## Detect Credential Dumping via reg.exe T1003.002

Adversaries may attempt to extract credential material from the Security Account Manager (SAM) database either through in-memory techniques or through the Windows Registry where the SAM database is stored. The SAM is a database file that contains local accounts for the host, typically those found with the net user command. Enumerating the SAM database requires SYSTEM level access. MITRE ATT&CK

In [1]:
%%spl2 -o aggrid
/* read attack dataset generated by Splunk Attack Range */
| from read_text("s3://smle-experiments/datasets/attack-range/T1003.002/attack_data.txt")
/* cast data as JSON */
| eval sysmon=from_json_object(cast(value, "string"))
/* read in the process name */
| eval process_name=ucast(map_get(sysmon, "Image"), "string", "") 
/* filter on cmd.exe and reg.exe */
| where process_name LIKE "%cmd.exe%" OR process_name="%reg.exe%"
/* read in the process name */
| eval cmd_line=ucast(map_get(sysmon, "CommandLine"), "string", "") 
/* filter by any command line string that has the word save and matches targetted registry */
| where cmd_line LIKE "%save%" 
AND (match_regex(cmd_line, /(?i)HKLM\\System/)=true 
OR match_regex(cmd_line, /(?i)HKEY_LOCAL_MACHINE\\Security/)=true 
OR match_regex(cmd_line, /(?i)HKEY_LOCAL_MACHINE\\SAM/)=true 
OR match_regex(cmd_line, /(?i)HKLM\\SAM/)=true 
OR match_regex(cmd_line, /(?i)HKLM\\Security/)=true);

 Finished.                     

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': […

# Now lets try a ML detection 

In [2]:
%%spl2 -o aggrid
/* read attack dataset generated by Splunk Attack Range */
| from read_text("s3://smle-experiments/datasets/attack-range/T1003.002/attack_data.txt")
/* cast data as JSON */
| eval sysmon=from_json_object(cast(value, "string"))
/* read in the different fields, time, process and command line */
| eval timestamp=parse_long(ucast(map_get(sysmon, "EventTime"), "string", null))
| eval process_name=ucast(map_get(sysmon, "Image"), "string", "")
| eval cmd_line=ucast(map_get(sysmon, "CommandLine"), "string", "") 
/* match on cmd.exe and extract the args */
| where process_name LIKE "%cmd.exe%" 
and match_regex(ucast(cmd_line, "string", ""), /.* \/[cC] .*/)=true
/* use first_time_event algorithm to determin the first time seen args */
| first_time_event cache_partitions=5 input_columns="cmd_line"
/* filter by the that were seen for the first time */
| where 'first_time-cmd_line';



 Finished.                     

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': […